# Fruit Identification Using Keras

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential, optimizers
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import sys
import numpy as np
from keras.preprocessing import image
import matplotlib.pyplot as plt

/home/nirali/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Image Augmentation

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('fruits/data/train/Banana/0_100.jpg')  
x = img_to_array(img)  
x = x.reshape((1,) + x.shape)  

i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='fruits/preview', save_prefix='banana', save_format='jpeg'):
    i += 1
    if i > 20:
        break  

# Part A Deep Learning model

In [3]:
# Building Model with 2 network layer, 1 fully connected layer and Output layer


def buildModel(activation,loss, optimizer):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(100, 100,3), activation = activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3), activation = activation))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation = activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(units = 128, activation = activation))
    model.add(Dropout(0.5))

    model.add(Dense(units = 9, kernel_initializer='zeros', activation = 'softmax'))

    model.compile(loss= loss, optimizer= optimizer, metrics=['accuracy'])
    return model

In [4]:
def buildComplexModel(activation,loss, optimizer):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(100, 100,3), activation = activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3), activation = activation))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation = activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), activation = activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), activation = activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(units = 128, activation = activation))
    model.add(Dropout(0.5))

    model.add(Dense(units = 9, activation = 'softmax'))

    model.compile(loss= loss, optimizer= optimizer, metrics=['accuracy'])
    return model

In [5]:
model = buildModel('relu','categorical_crossentropy','rmsprop') 
#model = buildModel('elu','categorical_crossentropy','rmsprop')
#model = buildModel('elu','mean_squared_error','rmsprop')
#model = buildModel('relu','mean_squared_error','rmsprop')
#model = buildModel('relu','categorical_crossentropy','ADAM')
#model = buildComplexModel('relu','categorical_crossentropy','ADAM')

#model = buildModel('relu','categorical_crossentropy','ADAM')




In [6]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('fruits/data/train',
        target_size=(100, 100), 
        batch_size=batch_size,
        class_mode='categorical') 

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory('fruits/validation',
        target_size=(100, 100),
        batch_size=batch_size,
        class_mode='categorical')

Found 4169 images belonging to 9 classes.
Found 1393 images belonging to 9 classes.


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 64)        18496     
__________

In [8]:
#fitting the model on training data
def fitModel(epochs):
    model.fit_generator(train_generator,
    steps_per_epoch=4361 // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=1460 // batch_size)

In [9]:
fitModel(3)

Epoch 1/3
272/272 [==============================] - 168s 618ms/step - loss: 1.4686 - acc: 0.4109 - val_loss: 0.8073 - val_acc: 0.7529
Epoch 2/3
272/272 [==============================] - 122s 449ms/step - loss: 0.7330 - acc: 0.7089 - val_loss: 0.5170 - val_acc: 0.8452
Epoch 3/3
272/272 [==============================] - 167s 614ms/step - loss: 0.4607 - acc: 0.8222 - val_loss: 0.3742 - val_acc: 0.8633


The accuracy using RELU with 3 epochs and cost function is categorical_crossentropy and gradient estimation technique is rmsprop 0.8222

In [12]:
# testing the model
def testModel():
    imageId = 1
    appleCount = 0
    bananCount = 0
    apricotCount = 0

    fig=plt.figure()

    for i in range(11):

        y = fig.add_subplot(3,4,i+1)

        path = "fruits/single_prediction/" + str(imageId) + "_100.jpg"
        imageId += 1 
        test_image = image.load_img(path, target_size = (100, 100))
        test_image = image.img_to_array(test_image)

        orig = test_image/255
        orig_1 = orig

        test_image = np.expand_dims(test_image, axis = 0)    
        result = model.predict(test_image)
    #     print(result[0])
        train_generator.class_indices

        if result[0][0] or result[0][1] or result[0][2] or result[0][3] or result[0][4] or result[0][5] or result[0][6] == 1:
            str_label = 'Apple'
            appleCount += 1
        elif result[0][7] == 1:
            str_label = 'Banana'
            apricotCount += 1
        else:
            str_label = 'Lemon'
            bananCount += 1

        y.imshow(orig_1)
        plt.title(str_label)
        y.axes.get_xaxis().set_visible(False)
        y.axes.get_yaxis().set_visible(False)
    plt.show()

# Part B Activation Function

In [13]:
model = buildModel('elu','categorical_crossentropy','rmsprop')

In [14]:
fitModel(3)

Epoch 1/3
272/272 [==============================] - 128s 469ms/step - loss: 1.1178 - acc: 0.5524 - val_loss: 0.8356 - val_acc: 0.7155
Epoch 2/3
272/272 [==============================] - 152s 558ms/step - loss: 0.5269 - acc: 0.7976 - val_loss: 1.2479 - val_acc: 0.7231
Epoch 3/3
272/272 [==============================] - 169s 620ms/step - loss: 0.3734 - acc: 0.8652 - val_loss: 0.5175 - val_acc: 0.8286


The accuracy using ELU with 3 epochs and cost function is categorical_crossentropy and gradient estimation technique is rmsprop 0.8652

# Part C Cost Function

In [15]:
model = buildModel('relu','mean_squared_error','rmsprop')

In [16]:
fitModel(3)

Epoch 1/3
272/272 [==============================] - 136s 501ms/step - loss: 0.0704 - acc: 0.4664 - val_loss: 0.0546 - val_acc: 0.6648
Epoch 2/3
272/272 [==============================] - 160s 589ms/step - loss: 0.0461 - acc: 0.7118 - val_loss: 0.0329 - val_acc: 0.8314
Epoch 3/3
272/272 [==============================] - 163s 599ms/step - loss: 0.0269 - acc: 0.8356 - val_loss: 0.0108 - val_acc: 0.9375


The accuracy using RELU with 3 epochs and cost function is mean_squared_error and gradient estimation technique is rmsprop 0.8356


In [17]:
model = buildModel('relu','mean_squared_error','rmsprop')

In [18]:
fitModel(3)

Epoch 1/3
272/272 [==============================] - 163s 600ms/step - loss: 0.0744 - acc: 0.4237 - val_loss: 0.0619 - val_acc: 0.5219
Epoch 2/3
272/272 [==============================] - 140s 515ms/step - loss: 0.0431 - acc: 0.7246 - val_loss: 0.0393 - val_acc: 0.7571
Epoch 3/3
272/272 [==============================] - 143s 525ms/step - loss: 0.0276 - acc: 0.8333 - val_loss: 0.0259 - val_acc: 0.8383


The accuracy using ELU with 3 epochs and cost function is mean_squared_error and gradient estimation technique is rmsprop 0.8333- All apples predcted as lemon

# Part D Epochs

In [19]:
model = buildModel('relu','categorical_crossentropy','rmsprop')

In [20]:
fitModel(10)

Epoch 1/10
272/272 [==============================] - 143s 526ms/step - loss: 1.2653 - acc: 0.4784 - val_loss: 1.1247 - val_acc: 0.4025
Epoch 2/10
272/272 [==============================] - 164s 604ms/step - loss: 0.6109 - acc: 0.7653 - val_loss: 0.2981 - val_acc: 0.9181
Epoch 3/10
272/272 [==============================] - 199s 730ms/step - loss: 0.4181 - acc: 0.8447 - val_loss: 0.4543 - val_acc: 0.7731
Epoch 4/10
272/272 [==============================] - 175s 643ms/step - loss: 0.3033 - acc: 0.8998 - val_loss: 0.2631 - val_acc: 0.9375
Epoch 5/10
272/272 [==============================] - 175s 642ms/step - loss: 0.2303 - acc: 0.9265 - val_loss: 0.2092 - val_acc: 0.8966
Epoch 6/10
272/272 [==============================] - 152s 559ms/step - loss: 0.1777 - acc: 0.9447 - val_loss: 0.2349 - val_acc: 0.9306
Epoch 7/10
272/272 [==============================] - 162s 595ms/step - loss: 0.1571 - acc: 0.9515 - val_loss: 0.1320 - val_acc: 0.9549
Epoch 8/10
272/272 [============================

The accuracy using RELU with 3 epochs and cost function is categorical_crossentropy and gradient estimation rmsprop technique using number of layers 6 and epochs 10 is 0.96


# Part E Gradient Estimation

In [21]:
model = buildModel('relu','categorical_crossentropy','ADAM')

In [22]:
fitModel(3)

Epoch 1/3
272/272 [==============================] - 162s 595ms/step - loss: 1.3911 - acc: 0.3964 - val_loss: 0.8237 - val_acc: 0.6107
Epoch 2/3
272/272 [==============================] - 152s 558ms/step - loss: 0.8324 - acc: 0.6373 - val_loss: 0.4731 - val_acc: 0.8175
Epoch 3/3
272/272 [==============================] - 152s 560ms/step - loss: 0.5549 - acc: 0.7766 - val_loss: 0.4140 - val_acc: 0.8411


The accuracy using RELU with 3 epochs and cost function is categorical_crossentropy and gradient estimation ADAM technique  and epochs 3 is 0.776

# Part F Network Arcitechture

In [25]:
model = buildComplexModel('relu','categorical_crossentropy','ADAM')

In [26]:
fitModel(3)

Epoch 1/3
272/272 [==============================] - 176s 647ms/step - loss: 1.5914 - acc: 0.3371 - val_loss: 0.9393 - val_acc: 0.6988
Epoch 2/3
272/272 [==============================] - 175s 645ms/step - loss: 0.9018 - acc: 0.6010 - val_loss: 0.8830 - val_acc: 0.5212
Epoch 3/3
272/272 [==============================] - 164s 603ms/step - loss: 0.6311 - acc: 0.7272 - val_loss: 0.5132 - val_acc: 0.7772


The accuracy using RELU with 3 epochs and cost function is categorical_crossentropy and gradient estimation ADAM technique  and layers 6 and epochs 3 is 0.776